In [49]:
import math
import pandas as pd
#variables

#####VARIABLES#######
boltDia = 0.625
shearStrength = 68 #this is the value for an A325 bolt with threads not in the shear plane 
numBolts = 3
boltSpacing = 3
e = 4  #this is the distance from the primary beam face to the CG of the bolt group

In [3]:
boltArea = (boltDia/2)**2*math.pi
phi=0.75 #this is the safety factor for bolt shear for the LRFD method
R_ult = round(phi*boltArea*shearStrength,3) # this is the design shear strength of a single bolt

print(f"The desing strength of a single bolt is: {R_ult}")
print("This value will be used as R_ult\n")

deltaMax = 0.34 #maximum bolt deformation per AISC 7-13

#determine location of bolts
bgHeight = boltSpacing * (numBolts-1)

boltCord = [] #starting from the top bolt

i = 0 
print(f"bolt coordinates with the y value of the center of the bolt group CG being 0:")
while i < numBolts:
    boltTuple = (0,bgHeight/2-boltSpacing*i)
    boltCord.append(boltTuple)
    print(f"bolt: {i+1} = {boltTuple}")
    i+=1


The desing strength of a single bolt is: 15.647
This value will be used as R_ult

bolt coordinates with the y value of the center of the bolt group CG being 0:
bolt: 1 = (0, 3.0)
bolt: 2 = (0, 0.0)
bolt: 3 = (0, -3.0)


In [48]:
#initial Ix and Iy assume the IC is at the vertical center of the bolt group (y=0)



####variables to be used in the calculations
c = 0 #distance from the IC to the individual bolt
cMax = 0 #distance of bolt furtherst away from the IC
cx = 0 #x distance from IC to the individual bolt
cy = 0 #y distance from IC to the individual bolt
R = 0 #Calculated strength on individual bolt
Rx = 0 #Caluclated strength in the x direction of an individual bolt
Ry = 0 #Calculated strength in the y direction of an individual bolt
mIC = 0 #moment value of the bolts around IC
sumMIC = 0 #total mIC for all bolts
delta = 0 #bolt deformation of individual bolt
deltaMax = 0.34 #maximum deformation of bolt before failur from AISC manual
icx = 0.5 #variable x coordinate for the IC
icy = 0  #this will be 0 for all vertical loads because we centered our bolt group vertically to our IC
ic = (icx,icy) #coordinates to the IC
p_ulty = 0 #total vertical load the bolt group can resist 
mP = 0 #moment calculated shear strength (p_ulty) of the bolt group around IC


iterations = 500 #number of times trying to find ic
iteration = 0
while iteration<iterations:
    cList = []
    for bolt in boltCord:
        c = math.dist(ic,bolt)
        cList.append(c)

    cMax = max(cList)

    #locations in bolt data list for each type of data

    boltData = pd.DataFrame(columns=["Bolt Number",
                                     "c",
                                     "cx",
                                     "cy",
                                     "delta",
                                     "R",
                                     "rx",
                                     "ry",
                                     "mIC"])
    
    boltNum = 1  #we don't want a bolt number 0
    for bolt in boltCord:
        c = math.dist(ic,bolt) #distance from the IC to the individual bolt
        cx = icx
        cy = bolt[1] #equal to the y value of the bolt coordinate
        
        delta = deltaMax*(c/cMax) #the delta of an individual bolt is linearly relative to deltaMax and cMax 
        
        R = R_ult*math.pow(1-math.pow(math.e,-10*delta),0.55)

        
        rx = -R*(cy/c) 
        ry = R*(cx/c)

        mIC = -rx*cy+ry*cx #get the moment of bolt reactions aroudn the IC

        boltData.loc[boltNum-1]=[int(boltNum),round(c,2),round(cx,2),round(cy,2),round(delta,2),round(R,2),round(rx,2),round(ry,2),round(mIC,2)]
        boltNum+=1


    sumMIC = boltData["mIC"].sum()
    p_ulty = boltData["ry"].sum() 
    mP = (e+icx)*p_ulty
    
    mDiff = abs(sumMIC-mP)   #the difference in the moment of the bolt reactions around IC and the Load around IC

    #convergence = abs(mDiff)/max(abs(sumMIC),abs(mP))
    #print(f"convergence = {convergence*100}%")
    print(f"difference in moment = {mDiff}")
    if round(sumMIC,2) == round(mP,2):
        print("solution found")
        iteration = iterations
    elif sumMIC>mP:
        icx+=mDiff*0.001
    else:
        icx-=mDiff*0.001
        
    #print("\n")
    #print(boltData)
    #print(f"iteration: {iteration}")
    #print(f"Total moment of bolt reactions around the IC = {round(sumMIC,2)}")
    #print(f"Total shear force the bolt groud can resist = {round(p_ulty,2)}")
    #print(f"Moment from the calculated shear strength of the calculation aroudn IC = {round(mP,2)}")
    #print("\n\n\n")
            
    iteration+=1

print(boltData)
print(f"Total moment of bolt reactions around the IC = {round(sumMIC,2)}")
print(f"Total shear force the bolt groud can resist = {round(p_ulty,2)}")
print(f"Moment from the calculated shear strength of the calculation aroudn IC = {round(mP,2)}")
    
        

difference in moment = 31.50500000000001
difference in moment = 27.727536049999998
difference in moment = 24.425555250849
difference in moment = 21.457695535017265
difference in moment = 18.8779158369544
difference in moment = 16.684916815419157
difference in moment = 14.69507229874904
difference in moment = 12.963320483989477
difference in moment = 11.456826936298384
difference in moment = 10.098164448012994
difference in moment = 8.897213925066552
difference in moment = 7.75916005868018
difference in moment = 6.8554185858038466
difference in moment = 6.105839520033371
difference in moment = 5.376883987774605
difference in moment = 4.6886954997614225
difference in moment = 4.17280933376135
difference in moment = 3.5996181039661224
difference in moment = 3.2547131490972845
difference in moment = 2.8219361248604287
difference in moment = 2.5477573909549562
difference in moment = 2.1743399457485992
difference in moment = 1.931224302395023
difference in moment = 1.6932692170947945
differe

In [17]:
#initial Ix and Iy assume the IC is at the vertical center of the bolt group (y=0)
icx = 0.5

iterations = 100 #number of times trying to find ic
iteration = 0
while iteration<iterations:
    ic = (icx,0) #y if IC is always 0 if the force is vertical
    
    #get a list of distances from each bolt to the IC
    distList = []
    for point in boltCord:
        distList.append(math.dist(ic,point))
    
    maxDist = max(distList)
    
    deltaList = []
    for dist in distList:
        deltaList.append(0.34*(dist/maxDist))
    
    rList = []
    for delta in deltaList:
        rList.append(R_ult*math.pow(1-math.pow(math.e,-10*delta),0.55))
    
    x = 0
    rxList = []
    ryList = []
    mList = [] #list of moments around IC
    for R in rList:
        boltPoint = boltCord[x]
        cx = boltPoint[0]-ic[0]
        cy = boltPoint[1]-ic[1]
        c = distList[x]
        rx = -R*(cy/c)
        ry = R*(cx/c)
        m = -rx*cy+ry*cx
        rxList.append(rx)
        ryList.append(ry)
        mList.append(m)
        x+=1
    
    i=0
    rxSum=0
    rySum=0
    mSum=0
    while i<numBolts:
        rxSum+=rxList[i]
        rySum+=ryList[i]
        mSum+=mList[i]
        i+=1
    
    mSum = round(mSum,2)
    print(f"the sum of bolt moments: {mSum}")
    ex = cg_x + ic[0] #distance from the IC to the applied load
    m_ult = -rySum*ex
    m_ult = round(m_ult,2)
    print(f"the moment of the resultant force around the IC: {m_ult}")
    m_diff = mSum-m_ult
    print(f"IC location: {ic}")
    if m_ult == mSum:
        print("solution found")
        iteration = iterations
    elif mSum>m_ult:
        icx+=m_diff*0.1
    else:
        icx-=m_diff*0.1

    iteration+=1

print(f"resultant reaction force: {-rySum}")
print(f"sum of rlist: {sum(rList)}")
print(f"nominal group strenth: {R_nom}")
r2List = []
for delta in deltaList:
     r2List.append(R_ult*numBolts*math.pow(1-math.pow(math.e,-10*delta),0.55))
print(r2List)

the sum of bolt moments: 98.32


<class 'NameError'>: name 'cg_x' is not defined